In [1]:
import psycopg2
import pandas as pd
import numpy as np
import datetime as dt
import os, sys, pickle, json, time, math, re
import mimicConcepts as mc

In [2]:
sqluser = 'yan'
dbname = 'mimic'
schema_name = 'mimiciii'
sqlpwd = '198609'
shost='localhost'
sport='5432'
con = psycopg2.connect(dbname=dbname, user=sqluser, password=sqlpwd, host=shost, port= sport)
cur = con.cursor()
cur.execute('SET search_path to ' + schema_name)
# Update the path for data extraction here
exportdir='/Users/yan/processed_data/'

In [ ]:
query = """
select c.hadm_id, c.icustay_id, c.itemid, d.label, c.charttime, c.valuenum
from chartevents c inner join d_items d on c.itemid = d.itemid
where c.itemid in (220050, 220051, 220052)
and c.hadm_id in (196611, 131076, 196617, 196619, 163856, 196633, 196655, 163893, 131128, 163899, 163902, 163904, 196674,
163913, 131151, 131152, 131162, 196702, 196705, 131171, 196710, 131180, 163951, 131184, 131186, 131193, 196733, 163965,
196735, 196741, 196746, 163980, 163982, 163985, 163987, 196762, 131239, 196779, 196782, 131246, 131262, 164032, 164037,
196809, 164041, 131281, 164055, 131289, 164058, 196828, 164062, 131297, 196840, 131306, 164074, 131308, 164079, 196851, 196852, 196855, 164088, 164091, 131324, 131325, 196867, 196870, 164104, 196881, 196883, 164119, 131353, 164132, 196906, 196916, 164156, 196931, 164200, 196970, 164212, 196984, 131449, 196985, 131457, 131468, 131471, 131472, 131473, 197013, 131484, 197023, 164259, 164262, 131495, 164269, 197039, 131510, 164278, 197051, 197055, 164295, 131530, 164300, 131550, 131551, 131563, 197100, 131570, 197106, 131577, 131578, 131584, 197122, 164360, 131593, 197131, 131600, 131608, 131613, 197150, 131618, 197164, 131634, 164419, 164425, 197193, 164431, 197203, 164444, 131679, 131681, 197222, 164462, 131710, 164479, 164480, 164486, 197255, 131721, 164491, 164496, 197268, 131733, 164504, 197276, 131740, 164508, 131746, 131754, 164527, 131774, 197310, 197313, 197319, 197321, 164556, 131795, 164588, 164591, 197360, 131825, 197368, 197372, 131840, 164609, 131845, 164637, 164649, 197418, 131884, 197423, 164657, 131890, 164664, 131907, 164676, 131910, 197450, 197451, 131921, 197461, 164693, 131928, 164699, 197470, 197473, 131946, 197483, 131948, 197487, 164722, 197492, 164728, 164733, 164735, 197506, 131973, 131979, 197532, 164768, 132017, 164786, 164791, 132029, 132034, 197570, 197571, 132042, 197579, 164814, 197584, 132050, 197591, 132059, 132065, 132066, 197606, 164846, 164854, 132089, 164862, 132104, 132115, 197656, 197658, 197659, 164894, 164897, 197665, 197668, 132140, 197680, 164918, 132153, 164940, 132172, 164945, 164948, 132180, 197736, 164981, 132218, 164987, 197756, 132223, 132226, 197773, 197778, 165011, 132244, 197785, 197786, 132255, 197793, 165028, 132265, 165037, 197810, 132286, 197823, 165060, 132293, 165074, 197850, 197851, 165089, 197858, 132321, 197862, 132328, 197869, 165107, 197876, 197875, 165121, 132357, 165136, 197916, 197917, 165157, 132389, 132393, 132398, 165171, 197939, 197946, 132410, 132412, 165181, 132414, 132413, 132418, 132423, 197966, 197970, 165208, 132443, 165214, 197984, 197985, 197991, 132458, 165232, 198009, 165242, 132473, 198012, 198013, 165246, 165252, 132485, 198022, 132488, 198033, 198036, 132507, 132509, 198046, 132513, 198058, 132522, 198067, 132531, 198070, 165310, 198083, 132548, 198093, 165329, 132575, 132582, 198124, 198125, 132588, 165366, 165373, 198143, 132612, 198152, 198156, 198158, 165392, 132634, 165409, 198184, 198187, 132664, 198208, 165445, 132678, 165465, 198238, 165473, 198252, 198255, 198262, 165503, 165505, 198275, 198282, 198291, 198296, 132768, 165536, 132776, 100009, 198314, 100011, 100012, 100016, 165553, 165554, 132786, 100024, 165566, 165569, 100034, 100035, 198340, 198342, 132812, 198349, 100047, 198352, 198353, 165586, 165594, 100063, 165600, 100068, 165605, 132840, 165610, 100075, 132850, 100085, 132861, 100102, 100104, 165648, 100116, 132889, 132890, 165660, 165667, 100133, 198439, 100138, 100141, 165689, 198458, 198463, 165700, 100165, 100166, 165704, 165710, 165719, 198491, 165724, 100188, 132955, 132960, 165732, 132967, 132974, 100208, 198512, 100210, 100215, 198523, 165757, 198528, 100227, 132998, 198537, 198538, 198545, 100250, 198556, 100263, 165804, 100271, 100286, 165829, 165830, 198600, 198603, 165841, 133083, 100320, 198626, 100325, 133094, 133095, 133098, 165867, 165868, 198636, 198638, 165870, 100337, 133107, 165878, 133116, 133120, 165894, 100361, 198665, 198668, 133145, 198686, 133150, 165922, 198693, 100394, 165933, 100399, 133168, 198706, 165940, 100406, 133176, 198713, 133177, 100415, 100416, 198724, 198726, 100423, 133193, 198734, 100442, 100445, 100449, 133221, 133222, 100453, 198760, 100463, 133240, 198779, 100477, 166015, 166017, 166019, 100485, 166023, 100492, 166034, 133269, 166058, 133292, 166062, 133298, 166068, 198839, 198841, 133307, 166079, 198856, 166092, 133326, 133332, 198874, 100571, 198882, 100579, 166117, 133349, 100583, 198890, 198899, 198906, 100606, 166144, 198917, 133382, 166149, 166152, 100619, 100622, 133392, 133400, 100636, 198942, 166175, 166187, 133422, 198959, 133428, 198968, 133442, 133444, 133456, 133457, 133463, 100696, 100697, 100699, 166236, 133473, 100713, 166249, 100715, 133483, 133485, 199019, 199017, 133491, 133492, 166261, 133496, 100730, 100732, 199044, 199046, 166282, 133514, 199052, 100749, 133518, 199054, 100756, 199066, 133537, 166309, 166313, 100779, 133553, 133559, 166330, 100795, 133566, 133570, 166340, 166341, 100808, 199113, 133577, 199117, 166355, 199125, 100827, 199131, 133597, 166370, 166373, 133622, 133633, 133638, 166410, 133643, 100877, 199183, 100881, 199190, 133654, 100890, 199206, 166450, 166453, 199224, 133692, 199229, 199232, 199235, 166467, 100934, 133708, 100943, 199248, 133711, 166492, 199264, 133737, 100977, 166514, 199292, 199296, 100999, 101006, 101011, 101018, 166555, 133788, 166563, 133798, 101033, 101034, 166573, 199342, 101039, 166577, 133810, 101046, 101047, 133814, 133826, 133827, 199366, 101063, 101065, 166602, 133835, 166616, 101085, 166622, 101092, 101104, 199409, 166642, 101110, 166646, 101113, 199418, 101117, 199426, 101124, 166665, 101131, 166670, 133904, 133915, 166686, 166696, 166698, 133942, 133950, 199487, 166722, 133956, 166725, 199500, 166737, 101206, 133975, 133982, 199518, 166752, 133991, 166760, 199528, 199530, 199533, 101244, 101245, 166780, 166788, 199562, 199565, 101264, 199574, 166808, 199577, 101277, 134053, 166823, 134057, 166832, 101302, 199609, 166849, 101317, 134087, 101322, 199627, 199628, 199631, 199633, 199643, 134107, 101343, 101344, 199651, 199655, 199659, 101361, 166898, 101372, 166909, 134143, 134147, 166916, 101382, 134151, 101388, 199699, 166934, 199702, 101401, 101405, 166942, 134176, 166952, 134184, 134185, 134187, 101422, 166964, 101428, 199734, 101430, 101432, 199740, 199742, 166982, 199751, 166987, 199756, 199757, 199762, 166995, 134230, 199769, 134233, 134239, 167010, 134244, 101481, 134250, 199789, 199791, 167030, 134267, 101501, 134270, 199812, 199813, 101514, 101517, 134286, 134287, 134288, 101521, 101525, 101528, 199834, 101537, 199846, 101543, 167079, 134313, 199856, 199859, 167095, 101560, 167098, 199867, 134342, 134344, 167116, 167125, 134359, 134366, 199909, 167150, 167160, 199928, 134396, 167164, 199940, 167175, 134410, 101643, 199948, 134419, 101657, 199962, 101668, 199972, 134439, 167207, 199979, 167226, 167227, 101695, 134467, 167237, 101705, 101706, 167244, 134479, 134482, 101715, 101727, 101742, 101748, 134520, 101756, 134528, 134537, 167314, 134554, 101787, 101797, 134566, 167344, 101816, 134586, 101818, 101821, 101824, 167366, 134600, 134607, 101851, 167391, 167400, 134632, 101866, 101872, 101876, 134648, 134653, 134656, 167427, 134661, 167431, 167435, 101905, 101906, 134680, 101914, 134684, 167456, 101921, 101922, 167459, 134699, 134704, 101945, 134713, 167483, 167484, 134719, 134720, 134739, 134740, 167515, 134762, 134768, 167539, 102018, 102019, 134792, 134799, 102033, 134801, 167581, 167584, 134826, 134830, 167598, 102063, 134835, 102069, 167609, 102074, 134842, 134854, 134857, 102089, 134867, 102103, 134885, 102120, 134890, 102127, 102128, 167664, 102135, 167675, 102141, 102142, 102148, 167687, 134920, 134923, 167697, 102165, 167714, 134946, 167725, 102193, 167729, 102195, 134966, 167747, 134985, 102223, 167762, 134998, 102237, 102242, 167781, 135018, 167788, 135023, 167791, 135028, 167797, 102272, 167809, 167810, 135044, 135047, 135053, 167823, 102295, 102298, 135073, 167854, 102326, 135095, 102327, 102333, 135102, 167876, 167879, 167883, 135117, 167897, 102365, 135138, 135143, 135151, 102385, 167932, 102396, 135168, 102405, 102408, 102412, 167951, 135200, 135204, 167972, 135206, 167977, 102444, 102445, 135216, 167986, 102456, 102459, 167996, 167998, 167999, 102467, 135244, 102480, 135249, 168021, 135254, 168026, 135261, 168035, 168044, 135280, 168055, 135290, 102523, 135295, 102529, 102536, 135304, 168075, 102546, 102558, 135329, 102563, 102565, 135334, 168106, 102575, 135343, 168113, 102576, 168125, 102591, 135364, 135365, 168133, 102602, 102603, 135373, 102605, 168144, 102616, 102625, 168161, 102629, 135402, 168173, 135407, 102641, 135411, 135414, 168183, 135417, 102649, 168187, 168189, 135425, 168197, 102667, 168205, 168210, 135448, 102683, 168223, 102695, 168232, 135479, 102713, 168252, 102725, 168262, 168288, 168291, 168296, 168308, 102774, 135549, 102783, 102785, 135554, 168326, 135559, 168334, 102805, 135574, 135578, 135595, 168364, 168369, 102835, 168375, 135614, 168385, 135632, 135633, 135644, 102877, 135659, 135664, 135668, 168437, 135672, 102906, 102916, 168460, 102928, 102929, 102932, 168470, 135707, 168477, 102942, 102944, 168480, 102952, 168495, 102964, 102965, 102966, 102972, 168513, 102978, 135749, 168524, 102997, 168543, 135787, 103026, 168567, 168571, 103041, 168578, 168580, 135812, 168583, 103054, 135824, 168597, 103065, 103085, 135859, 168628, 103098, 168639, 135873, 103108, 135878, 103125, 103129, 168666, 103133, 135905, 135906, 103139, 103140, 135913, 168682, 103154, 168693, 103164, 103165, 103166, 168704, 135940, 135942, 135946, 168721, 135960, 168730, 135968, 103203, 103209, 168745, 103211, 168753, 135985, 168759, 135993, 168761, 168766, 136013, 168788, 136021, 103258, 168800, 168802, 103269, 136039, 103276, 136044, 136047, 136051, 136056, 103289, 136058, 168827, 168824, 136064, 168833, 136071, 103309, 168849, 103315, 136085, 103319, 136087, 168863, 136099, 103349, 103355, 168897, 168904, 103381, 168919, 136152, 168925, 103389, 103429, 168974, 168977, 136210, 136211, 103446, 168985, 136219, 136220, 168993, 136231, 103473, 103478, 103480, 136264, 103499, 103500, 136273, 169055, 136288, 169062, 169066, 169067, 103532, 136301, 103539, 136328, 169097, 136332, 136339, 103583, 103584, 136366, 103601, 136370, 169146, 169151, 103622, 136391, 169165, 103630, 103646, 103650, 136419, 136423, 103656, 136425, 136424, 103677, 169213, 103692, 169229, 103694, 169230, 103704, 169240, 103706, 103713, 136481, 169252, 169253, 103726, 169271, 169273, 136506, 136507, 103741, 169281, 169287, 136542, 169320, 136560, 103793, 169331, 169333, 103810, 136583, 103815, 136589, 169364, 136600, 103841, 169386, 169387, 169397, 169401, 136634, 169408, 136645, 169414, 169416, 103881, 103884, 103885, 103886, 136655, 136659, 103892, 136665, 169436, 103901, 136674, 136676, 136680, 169449, 169453, 103921, 103922, 169457, 136693, 169464, 103933, 136704, 103937, 103944, 103945, 103947, 136717, 169488, 136725, 169499, 136735, 169506, 103979, 103989, 169527, 136762, 169531, 104002, 104005, 104006, 104009, 136786, 104023, 169560, 136802, 104040, 136810, 169579, 104047, 104052, 104058, 104060, 104073, 136842, 136849, 136852, 104084, 104086, 104090, 136860, 169630, 104094, 136870, 104110, 104112, 169649, 136884, 169654, 104122, 136892, 104127, 136896, 169667, 136906, 104140, 104145, 104147, 136923, 104157, 169694, 169705, 169708, 104178, 169730, 169731, 169733, 104203, 169757, 169763, 104234, 104235, 169771, 137008, 104243, 169792, 104262, 169808, 137041, 137043, 104287, 137057, 137059, 137065, 169834, 169836, 169837, 169845, 104312, 104319, 137087, 137094, 137103, 169880, 104348, 137119, 137122, 169894, 137131, 137133, 104366, 169907, 169927, 104395, 104398, 169935, 104400, 137169, 169944, 104410, 137196, 137200, 104434, 169972, 137211, 169980, 169994, 137226, 137233, 104465, 137240, 137247, 170020, 104485, 170023, 170024, 170039, 137277, 104520, 170057, 104524, 170066, 170069, 104538, 170075, 137307, 104549, 137343, 170112, 104575, 137346, 104580, 170117, 137350, 170125, 104602, 104605, 104613, 104619, 170157, 170163, 170165, 137406, 104647, 137417, 137430, 170204, 170206, 170208, 104680, 104681, 104687, 137463, 104696, 137467, 170239, 104704, 137472, 137479, 170249, 137488, 104721, 104723, 170269, 137504, 137516, 170292, 104759, 170297, 170306, 137539, 170308, 170312, 104798, 104799, 170337, 104808, 170348, 104814, 137592, 104829, 170371, 170376, 137609, 170377, 170389, 170400, 137634, 170406, 104877, 137650, 170418, 137658, 137664, 137668, 104900, 170450, 170465, 104929, 137701, 104935, 170472, 104950, 170491, 104957, 170494, 170495, 170505, 137739, 170511, 137744, 170514, 170515, 104979, 137750, 104985, 170526, 105016, 105017, 137786, 137788, 170566, 170569, 137808, 105042, 137810, 170580, 170587, 170597, 137830, 137834, 137839, 170609, 137842, 105075, 105080, 105084, 170621, 137855, 170625, 137862, 137866, 137869, 170640, 105123, 137902, 105135, 137906, 170675, 105139, 105146, 137917, 137919, 170691, 105160, 137934, 170704, 170710, 137946, 137949, 137952, 105189, 105199, 170747, 105217, 170754, 137985, 137992, 170760, 170762, 105237, 105240, 170782, 170786, 170789, 170794, 138030, 170802, 138038, 138039, 138041, 170811, 138046, 170816, 105280, 105286, 138057, 170826, 170830, 105310, 170849, 138082, 105316, 105328, 138099, 138100, 138103, 170875, 105341, 138114, 170884, 138116, 105351, 138121, 170894, 138127, 105362, 138132, 105369, 138144, 138145, 105381, 105391, 105393, 170932, 138182, 105417, 105418, 170959, 138192, 105429, 105430, 170972, 105449, 138218, 138219, 105457, 170994, 138225, 105464, 171001, 105468, 105473, 171013, 171020, 171024, 105497, 105504, 171040, 105506, 171051, 105519, 171057, 138290, 138297, 105534, 138302, 138310, 105543, 171079, 105547, 138322, 171095, 171097, 105566, 138344, 105582, 171121, 171127, 138363, 105605, 171141, 171144, 171145, 138379, 105625, 171174, 105638, 171177, 138410, 105648, 171190, 105655, 171192, 105661, 171197, 171204, 138437, 105671, 171208, 138440, 138446, 171219, 171226, 105697, 105708, 138477, 138481, 138482, 171251, 138484, 105718, 105720, 138495, 138506, 105743, 171280, 171293, 105768, 138541, 105780, 105790, 105804, 171342, 105807, 105806, 138578, 171357, 138598, 105830, 171369, 171372, 171380, 138617, 138623, 171393, 138635, 171405, 105872, 171418, 138651, 105883, 105895, 105900, 105911, 105928, 138697, 138705, 171476, 105948, 138719, 105952, 105953, 105956, 171516, 105981, 138750, 138752, 105987, 105989, 105990, 105994, 138764, 171536, 171554, 106021, 138790, 106028, 171568, 138801, 138802, 171573, 106042, 171586, 106050, 138823, 138824, 171597, 138831, 138834, 171612, 106093, 138866, 138877, 171645, 106110, 106112, 171649, 138881, 171674, 138914, 138915, 106152, 138935, 138938, 138940, 138943, 138947, 138960, 171730, 138967, 171736, 106200, 171735, 171753, 138985, 106225, 106229, 138998, 138999, 171768, 106231, 106234, 171787, 171794, 106258, 139029, 171802, 106269, 171807, 171816, 106288, 106294, 139066, 139067, 139070, 106305, 106306, 106307, 106308, 139078, 106315, 139092, 139099, 106332, 139100, 106336, 171873, 106350, 171890, 106359, 139129, 139146, 106390, 139161, 106393, 106398, 106415, 139184, 171951, 106421, 171965, 139199, 106443, 171985, 139218, 106452, 139221, 171991, 139226, 139227, 139228, 106464, 139239, 106473, 106484, 139265, 172042, 106507, 172043, 139281, 106513, 139283, 139284, 106517, 172057, 139290, 172076, 172082, 106551, 106556, 172093, 106561, 172104, 172110, 106577, 139354, 172124, 106589, 172133, 139380, 139394, 106630, 172167, 139400, 139402, 106638, 106641, 172193, 139427, 106665, 106671, 106672, 106674, 139444, 172216, 172221, 106691, 139465, 139475, 139485, 139512, 172283, 139521, 106763, 139534, 172317, 139550, 172342, 139580, 172355, 172378, 172385, 139620, 172393, 172397, 106862, 172401, 139637, 172407, 139644, 106883, 172420, 139654, 106889, 139660, 172429, 106900, 172436, 172438, 172439, 172441, 139683, 172453, 139692, 172460, 139697, 172472, 106939, 139708, 172476, 172492, 139725, 172501, 172502, 139744, 172515, 139754, 139755, 106990, 139761, 172530, 106996, 172532, 172535, 139769, 107004, 107013, 172553, 172555, 172557, 139797, 139801, 107034, 139813, 107049, 172603, 172609, 139844, 107082, 139851, 172623, 139862, 172630, 172640, 107104, 172657, 107122, 139893, 107132, 139905, 107141, 139914, 107147, 172689, 139921, 139924, 139926, 139938, 139942, 172718, 139953, 107186, 139955, 172724, 107202, 172739, 107203, 107216, 107220, 107225, 140003, 107238, 140010, 172779, 107247, 140022, 107255, 107272, 172813, 172824, 172829, 107294, 140068, 172837, 140078, 172852, 140086, 140097, 172869, 107340, 140112, 140114, 140120, 107354, 107357, 140128, 107367, 172906, 140145, 107379, 172918, 107385, 107386, 107394, 107397, 107406, 172959, 172970, 172974, 140212, 140220, 140226, 140228, 107462, 140236, 140242, 107475, 173013, 140247, 107489, 140261, 173030, 107495, 107499, 107507, 140281, 173055, 140290, 107528, 173070, 140303, 107549, 173089, 173092, 140334, 107567, 107569, 107570, 140338, 173112, 107578, 107580, 140350, 140351, 140357, 140363, 140367, 140368, 173136, 140369, 173152, 173155, 107622, 107625, 140393, 107633, 140402, 140403, 107644, 173180, 173194, 173203, 107671, 140443, 140448, 107683, 140458, 107690, 107692, 173235, 140473, 107707, 107720, 140491, 140497, 173269, 107746, 173285, 140521, 140529, 140536, 140543, 140551, 173323, 173324, 107794, 173330, 173336, 173347, 107813, 140585, 107820, 173358, 140591, 140590, 140593, 107826, 107835, 107847, 140622, 140625, 107866, 107869, 140642, 140651, 173420, 140655, 173431, 140665, 173435, 140671, 173443, 140683, 173458, 140692, 140696, 140705, 140707, 107942, 140711, 107946, 173484, 173496, 173499, 107970, 173513, 140745, 173522, 140761, 107995, 173540, 173542, 140787, 140792, 108025, 108028, 173565, 140799, 108032, 173568, 108040, 108043, 140817, 173586, 108052, 173595, 140840, 108079, 108080, 173623, 173628, 140875, 173650, 173651, 108121, 108124, 173660, 173663, 108132, 108135, 140904, 140909, 140912, 173683, 108151, 140920, 108153, 173690, 140923, 140927, 108164, 173706, 173707, 173711, 108176, 173717, 108200, 108204, 140974, 140977, 173747, 173748, 140990, 108227, 173769, 141004, 173776, 141008, 141009, 108251, 141032, 141036, 173810, 173812, 108281, 173819, 108287, 173825, 173826, 141061, 141073, 141075, 173844, 108310, 173851, 141085, 173860, 173862, 173864, 108330, 108338, 108343, 173885, 141129, 141130, 108363, 141134, 141135, 141136, 108370, 141153, 141157, 108394, 141162, 108400, 141170, 173939, 173943, 173944, 173960, 108429, 108432, 141202, 108439, 173990, 141233, 108470, 141246, 174021, 141261, 141264, 174033, 174037, 141273, 174044, 108509, 174051, 174058, 174067, 141309, 141314, 141326, 141336, 174107, 141343, 108588, 174124, 108594, 141363, 108597, 174134, 108602, 174140, 174141, 174153, 174161, 174162, 174163, 174164, 108640, 141412, 174191, 141425, 174197, 141440, 141446, 108679, 141450, 108685, 141453, 141455, 141459, 108704, 174243, 174246, 108712, 141483, 174253, 141494, 141512, 141515, 108752, 108754, 174308, 108772, 108782, 108789, 141566, 174335, 141569, 108809, 108810, 174348, 174356, 141588, 174359, 108831, 108832, 108835, 141612, 174380, 108847, 174384, 174385, 108856, 174399, 108863, 141632, 108873, 141642, 174411, 174419, 108884, 174421, 141651, 108889, 174433, 141667, 141668, 174436, 108904, 108905, 108910, 141687, 141690, 174463, 174465, 141707, 174477, 174478, 141709, 174481, 174485, 108952, 174494, 108958, 174504, 174513, 174516, 141750, 141762, 108998, 141773, 174545, 174559, 174564, 174570, 109037, 141813, 174582, 174586, 141821, 109058, 174595, 174608, 174611, 174615, 109079, 174618, 141857, 109090, 141860, 109130, 109131, 109134, 141903, 109136, 109147, 141916, 109151, 141920, 141922, 174697, 174705, 109174, 174733, 109201, 109208, 141976, 141979, 174752, 174754, 141986, 109221, 141990, 174760, 109242, 109245, 109249, 109250, 174787, 142023, 109255, 174811, 109282, 142052, 174823, 109289, 109290, 174828, 142065, 109302, 109307, 109309, 109313, 174850, 109316, 174861, 109330, 142103, 109341, 174877, 142122, 174892, 109359, 174896, 109368, 109370, 174913, 174918, 142151, 174921, 109386, 174933, 109400, 142171, 109409, 142178, 142185, 109427, 109434, 174972, 109437, 174974, 142207, 109444, 109446, 109463, 175001, 175004, 109469, 109478, 142248, 175022, 109491, 109495, 175031, 142265, 175034, 175035, 175036, 142271, 109505, 175043, 109516, 142287, 109534, 142306, 109542, 175079, 142313, 142323, 109563, 142332, 109567, 142335, 109576, 142347, 142354, 175127, 175131, 142369, 175151, 109617, 109627, 175164, 175171, 142412, 175188, 142423, 142427, 109661, 175201, 109679, 109683, 109687, 175223, 175225, 109695, 109702, 175249, 142486, 175258, 109722, 175262, 109727, 142497, 175273, 109737, 142507, 175284, 109750, 142520, 109752, 109757, 142534, 175319, 142551, 142555, 142565, 109803, 175341, 142575, 175344, 175345, 142584, 142586, 175366, 109831, 109832, 175372, 175375, 142625, 175401, 109866, 109874, 175410, 109878, 142652, 109893, 175432, 109900, 142671, 175440, 109905, 142677, 175445, 142680, 109931, 109932, 142707, 142709, 175479, 142712, 175481, 175483, 142724, 142727, 175506, 175510, 142743, 109975, 109985, 142755, 175524, 109999, 175537, 110002, 175541, 142773, 175550, 110032, 110035, 110058, 175595, 175597, 175598, 175601, 175602, 110075, 142850, 142856, 175627, 110099, 110101, 175638, 110115, 110136, 110137, 175673, 142911, 110153, 110157, 142927, 175700, 175701, 175706, 142945, 110178, 175724, 142958, 110191, 142965, 175736, 175738, 142977, 142978, 175746, 142986, 175756, 142995, 110229, 110230, 142997, 175766, 110234, 175773, 175780, 143017, 175792, 143026, 110259, 110262, 175800, 143033, 175803, 110277, 110278, 143047, 175814, 110282, 143051, 143054, 175825, 110291, 110293, 143063, 175833, 175840, 175848, 110312, 110313, 110316, 110319, 110341, 110342, 110343, 175881, 175882, 110351, 110362, 143138, 143147, 143149, 110383, 110391, 175933, 110402, 143175, 110408, 175945, 175947, 143191, 143195, 110430, 110432, 143200, 175968, 175977, 143211, 110449, 175988, 175989, 143224, 110458, 143239, 176015, 176016, 110484, 110493, 143264, 143268, 110501, 176041, 110511, 143280, 110521, 143290, 176062, 143296, 110529, 110530, 176067, 176070, 110535, 143309, 176082, 110549, 143323, 110562, 110563, 110570, 143340, 110577, 110580, 143349, 143354, 110586, 110593, 176130, 176131, 143365, 176135, 110600, 110601, 176136, 143381, 110614, 110617, 176165, 110630, 176166, 176174, 143408, 143413, 143414, 176182, 110647, 110651, 143423, 110660, 143428, 143438, 143440, 176210, 110683, 143452, 110693, 110695, 143468, 176239, 143475, 143483, 143505, 110739, 176282, 176286, 143531, 110767, 176304, 143537, 143538, 143544, 110778, 110780, 143556, 176327, 176335, 176343, 143580, 110820, 143588, 110826, 143594, 110834, 110837, 176393, 176395, 143628, 143633, 176401, 176409, 176417, 176418, 110889, 143661, 143671, 110909, 110910, 143683, 110915, 176461, 110930, 176468, 176475, 176477, 110952, 110953, 110958, 143742, 176512, 176516, 176530, 111005, 143774, 111010, 111023, 176559, 143791, 176562, 176567, 111031, 143803, 176574, 111041, 111043, 111044, 143811, 111046, 143818, 176592, 111057, 111059, 143829, 143834, 143847, 111081, 111089, 176631, 143865, 143870, 111103, 176653, 176657, 176658, 143899, 111132, 176670, 111144, 176680, 143915, 111153, 111156, 111158, 143927, 111160, 176725, 143964, 143966, 176735, 176736, 111208, 143979, 176749, 176750, 111213, 111223, 143994, 176776, 111240, 144029, 111266, 111274, 176811, 111283, 111288, 176832, 176834, 144067, 176839, 111303, 176840, 176846, 111316, 144085, 111323, 176866, 176871, 111336, 144131, 111367, 111372, 111377, 144152, 176922, 111389, 111396, 144172, 176943, 144175, 176950, 176953, 111419, 144188, 111421, 144195, 144200, 176970, 176974, 176979, 111444, 176986, 144218, 176991, 111457, 144226, 144227, 144231, 144238, 177015, 177016, 177024, 144260, 111494, 177030, 177038, 177042, 177047, 111515, 177055, 111529, 177073, 177077, 144310, 177078, 144314, 144324, 111574, 144344, 111580, 177121, 144353, 177133, 177134, 111603, 177143, 111610, 111619, 144387, 177162, 144395, 177168, 177172, 177173, 111646, 144422, 144426, 111666, 177207, 144442, 144465, 111699, 177236, 144472, 144474, 111707, 111708, 177252, 177254, 111722, 177259, 144499, 111736, 177276, 144509, 144522, 177307, 177315, 144548, 177327, 177329, 144564, 177337, 177338, 177339, 177340, 177347, 144582, 144591, 144594, 144609, 144612, 111845, 111846, 144631, 111865, 111867, 144639, 144640, 144643, 144644, 111890, 177427, 144660, 144662, 144663, 177436, 177443, 111907, 144677, 144679, 177453, 177455, 111919, 111923, 111928, 111938, 111943, 111946, 111955, 177491, 111959, 111964, 111967, 144739, 111972, 111973, 111977, 144747, 144748, 111981, 177517, 177528, 177546, 112012, 177554, 144794, 112028, 177566, 144800, 177571, 144804, 177592, 144828, 177601, 177612, 144844, 144855, 112089, 112090, 177630, 177636, 177637, 144869, 177641, 112121, 112122, 144912, 144913, 177694, 177697, 112168, 177706, 144940, 177708, 177711, 177714, 177715, 112178, 144952, 144957, 144965, 177737, 112201, 112203, 144987, 112220, 112225, 144994, 177761, 112228, 144997, 112233, 112235, 145004, 177781, 177783, 112253, 145027, 112265, 177802, 112275, 112286, 177823, 145064, 177847, 112313, 145082, 145083, 112318, 112325, 177862, 177868, 112335, 112337, 112338, 177875, 112342, 177882, 177890, 145127, 112360, 145131, 112368, 177911, 177929, 177937, 177938, 112413, 177960, 112432, 177973, 145207, 145209, 112445, 145218, 145225, 145231, 112464, 145234, 145235, 145236, 178007, 112474, 112475, 178012, 145245, 178016, 145250, 145252, 112501, 178038, 145271, 112516, 112520, 145303, 112538, 145307, 178083, 145326, 178096, 112561, 145332, 178104, 178105, 145340, 178113, 112578, 178133, 112600, 145370, 145375, 145378, 145379, 112611, 112627, 178164, 145398, 145399, 178168, 112634, 112635, 145410, 145414, 145415, 112651, 145436, 178205, 145440, 178211, 178216, 112689, 178225, 145465, 145467, 112706, 145476, 178246, 112721, 145495, 178264, 178268, 112732, 112747, 145518, 178290, 178294, 112759, 178296, 112762, 112764, 178301, 112767, 145535, 145537, 112776, 178313, 112777, 178316, 112784, 145554, 145555, 112790, 112791, 178345, 112809, 112817, 145585, 145595, 112832, 178380, 112866, 178404, 178408, 145645, 112885, 112886, 145660, 178428, 145663, 178431, 145667, 178436, 145681, 178449, 178450, 145684, 145685, 178455, 145688, 178456, 112919, 145689, 178461, 112925, 112928, 145700, 178480, 112945, 112947, 145724, 145733, 178509, 145742, 145741, 178510, 178511, 178517, 112991, 178527, 113006, 113011, 113014, 178556, 145790, 178559, 178565, 113030, 178567, 113032, 145797, 178571, 178575, 178576, 113045, 178594, 145827, 113058, 113066, 113076, 145845, 113086, 145854, 145856, 178625, 145859, 113093, 145866, 113099, 178636, 178639, 113105, 178643, 178662, 113127, 178683, 178685, 113150, 178687, 178692, 113159, 145928, 145929, 178705, 178708, 178712, 113177, 113202, 178738, 178743, 178751, 113217, 178758, 145993, 145999, 113237, 178773, 178787, 178790, 146025, 178800, 113265, 178813, 146046, 113279, 113288, 146066, 146068, 146069, 178842, 146081, 146083, 113315, 113320, 113326, 146096, 113331, 113335, 178877, 113343, 178896, 146129, 113367, 178903, 146137, 146138, 178909, 113379, 146151, 178922, 146156, 113395, 146166, 113412, 178949, 178952, 113423, 178961, 146195, 178964, 113433, 178970, 146208, 113444, 146213, 178985, 113455, 146223, 113457, 146226, 113458, 178997, 146230, 113461, 179002, 113466, 113481, 146254, 179031, 179034, 179035, 179056, 146292, 146300, 113534, 113539, 113540, 179079, 146314, 179082, 146336, 146341, 113575, 179114, 146348, 179118, 146354, 146371, 146377, 146385, 179155, 113620, 146395, 179166, 113630, 179175, 113640, 179179, 179180, 179185, 113667, 113671, 146452, 113685, 113692, 113693, 113703, 113704, 179242, 146476, 179248, 146481, 146491, 179261, 146495, 146505, 179275, 179276, 146509, 113740, 113747, 179284, 113749, 179293, 146534, 146536, 113770, 113773, 146557, 113791, 179340, 146575, 179348, 113817, 146589, 179359, 146598, 146608, 146613, 113852, 179388, 113873, 146644, 113877, 179414, 113883, 146656, 146659, 179438, 113912, 146682, 179455, 146690, 179458, 113931, 146704, 146709, 179477, 146711, 179480, 146712, 113948, 113953, 146722, 146725, 113965, 146736, 146739, 179507, 113977, 179516, 113980, 179520, 113991, 179527, 146761, 113994, 179531, 146765, 179533, 179538, 114008, 179544, 179551, 114016, 179557, 146789, 114026, 179575, 179581, 114047, 146815, 146823, 114058, 146829, 114065, 146842, 146843, 146845, 146846, 179616, 114082, 179620, 114087, 146866, 179637, 114104, 146883, 179657, 114122, 179659, 146893, 179661, 146900, 179671, 179672, 179678, 179683, 179691, 179696, 179704, 179708, 179713, 179716, 146953, 146957, 146959, 114194, 179735, 179741, 146974, 114208, 146984, 146991, 179764, 114230, 114236, 179777, 147013, 114245, 147018, 114250, 179787, 179790, 114258, 179795, 147027, 147029, 179797, 147030, 114271, 147047, 179816, 114308, 147079, 114312, 147085, 179855, 114322, 179865, 147104, 114342, 147111, 179886, 179892, 147125, 114358, 179896, 147129, 179898, 147131, 147128, 147137, 114370, 147139, 179908, 114378, 114384, 147153, 179937, 147170, 179948, 147181, 179954, 114419, 114421, 147191, 179961, 114427, 114435, 114441, 179978, 179982, 114454, 179991, 114465, 147243, 114477, 180018, 180023, 180024, 180027, 147266, 114503, 114504, 147276, 180045, 147281, 114514, 180052, 147286, 114520, 147293, 114532, 147311, 147314, 147320, 147321, 114554, 114553, 147324, 180093, 114559, 147332, 114566, 180108, 114573, 180112, 180114, 180115, 114583, 180121, 180136, 114601, 114602, 147376, 180150, 147384, 114617, 114618, 147385, 114639, 114644, 180186, 114656, 114659, 114661, 180199, 147433, 147441, 114691, 147464, 180261, 180262, 147496, 147500, 180271, 114737, 147506, 114740, 180277, 147508, 147511, 114743, 180285, 180286, 147544, 147545, 114787, 180332, 147564, 180335, 147568, 147569, 180342, 147577, 114812, 114815, 114818, 180356, 147593, 180367, 114840, 114843, 147614, 147620, 114854, 147623, 114856, 114857, 114864, 114866, 147635, 114877, 180419, 114884, 147654, 180433, 114898, 147666, 114902, 114906, 180446, 180456, 180458, 114924, 147696, 147697, 114932, 147703, 114939, 147709, 114945, 114950, 147721, 180494, 147727, 180499, 147738, 114974, 180513, 114980, 114994, 114996, 147764, 180542, 115009, 180548, 180549, 147783, 180551, 115021, 115022, 180558, 180565, 115032, 147800, 180569, 180577, 147829, 180602, 180614, 115080, 115081, 147851, 115085, 180634, 147873, 115111, 115115, 147884, 115119, 115120, 147892, 180663, 147896, 180668, 147901, 147903, 115139, 147907, 180678, 180680, 180682, 147915, 115151, 115153, 115169, 180711, 147947, 180716, 147952, 115187, 180723, 115195, 180732, 147970, 180740, 115213, 180754, 115219, 115222, 115229, 180767, 180773, 148007, 115251, 180787, 115262, 115264, 180805, 148037, 180810, 180812, 115279, 148052, 180821, 180827, 148060, 180836, 115300, 180840, 115305, 148075, 148079, 148080, 180858, 148092, 180870, 148105, 115339, 180880, 148113, 115350, 115353, 148124, 115359, 180896, 148131, 148141, 180914, 180916, 115381, 115394, 180937, 115403, 180944, 148182, 180951, 115419, 180961, 115429, 148204, 115438, 180985, 115450, 115473, 181010, 181011, 115476, 115485, 181021, 181022, 115492, 115496, 148269, 181043, 181044, 115521, 181061, 115528, 181064, 115535, 115545, 181092, 115562, 148333, 148337, 181115, 181116, 115587, 148356, 148359, 115591, 148368, 148373, 148382, 181151, 148384, 181150, 181156, 181161, 115626, 181163, 181169, 181173, 148409, 148413, 181183, 148427, 115661, 181218, 181221, 148454, 115688, 181228, 115693, 181231, 148468, 181238, 115704, 181249, 148482, 148483, 181254, 148488, 181257, 148492, 115729, 115735, 181284, 181286, 181287, 115757, 148530, 148532, 148561, 115795, 181332, 181337, 148579, 181352, 181359, 115833, 181376, 115841, 115856, 181394, 181397, 148637, 181409, 148642, 181427, 115896, 115898, 181438, 115903, 181443, 181444, 181446, 148678, 115915, 148706, 181486, 115959, 148735, 115969, 181514, 115981, 181525, 148762, 148765, 115997, 148773, 148777, 181547, 116011, 148784, 181554, 181561, 148805, 181580, 116044, 148819, 148825, 116059, 181600, 181603, 148875, 181651, 181653, 148889, 148890, 181662, 148894, 116127, 148900, 148901, 116138, 116143, 148912, 181682, 148920, 148923, 181692, 148927, 148929, 116163, 148934, 148937, 148942, 116183, 181726, 181730, 148966, 148974, 181746, 116210, 181753, 116222, 116224, 116225, 181763, 116230, 148999, 181768, 149005, 181774, 116242, 181779, 181783, 116250, 181786, 116252, 116262, 181802, 181803, 181806, 149041, 181811, 116280, 116296, 149069, 116303, 149079, 149092, 116325, 181862, 116326, 149096, 149101, 149106, 149115, 181884, 149117, 149123, 181894, 116358, 149134, 116366, 181906, 116378, 116379, 181918, 116385, 149155, 116394, 181946, 149185, 181954, 181955, 116420, 149190, 149191, 181960, 116426, 149197, 181965, 149206, 116445, 116446, 149218, 116451, 181988, 181990, 149229, 149231, 182005, 149246, 149248, 182019, 182023, 116487, 116498, 182035, 116500, 116514, 116517, 116519, 116520, 182058, 149298, 116531, 182068, 116534, 182082, 182096, 149331, 182113, 182115, 116581, 149358, 182130, 149363, 149365, 149366, 182142, 116608, 116612, 182149, 116613, 149384, 116624, 182161, 182164, 149397, 182166, 116647, 149417, 116656, 182202, 116673, 149446, 116683, 182219, 149453, 182225, 182233, 182237, 149470, 116703, 149476, 116727, 149503, 149515, 116748, 149518, 149520, 182300, 116768, 149539, 116772, 149542, 116780, 116782, 182319, 116802, 182338, 149573, 182342, 149575, 116814, 182352, 182356, 116826, 149601, 182370, 182372, 149609, 182380, 116847, 149621, 182393, 116857, 182397, 116863, 149638, 182410, 149643, 116877, 182422, 116901, 182444, 182447, 182457, 149702, 116942, 182481, 182484, 182494, 149726, 182496, 149732, 149734, 182505, 149738, 116980, 149750, 149756, 116993, 182529, 116996, 182538, 182546, 117015, 149789, 117032, 149803, 149804, 117037, 117045, 149821, 149828, 182599, 149839, 149850, 117091, 149860, 149865, 182640, 117110, 117111, 117117, 117124, 117125, 182660, 182669, 182683, 149915, 182691, 149931, 182702, 149941, 149942, 182711, 182713, 149946, 182715, 182727, 182747, 182750, 117218, 149988, 149990, 150005, 150015, 182790, 182793, 117259, 117272, 117277, 150050, 182830, 117302, 117308, 182845, 117314, 150093, 182862, 117328, 150097, 182875, 117344, 182884, 150119, 117360, 150138, 150147, 150148, 150152, 117384, 150165, 182940, 117407, 182946, 117412, 117413, 150186, 182958, 117430, 150201, 117435, 150204, 150216, 182997, 117468, 183006, 117476, 117483, 117488, 150260, 117502, 183040, 150280, 150284, 117517, 150285, 117526, 117528, 150298, 117530, 183070, 117536, 117549, 183088, 150324, 150325, 183106, 150348, 183120, 150357, 183126, 183127, 150366, 117599, 150377, 183146, 117612, 150382, 150385, 183153, 117620, 117627, 150397, 183166, 117632, 117636, 183174, 183176, 117643, 117645, 183187, 117669, 150447, 183220, 117684, 150457, 183226, 183244, 183246, 150479, 150478, 150482, 183251, 150486, 150489, 117730, 150509, 150513, 117745, 183289, 183299, 150533, 150534, 183305, 183306, 150541, 117774, 183318, 150556, 117790, 150561, 150564, 117796, 150566, 117797, 117815, 183358, 117822, 117825, 183367, 183385, 183388, 183395, 183397, 117871, 117876, 150649, 150655, 150662, 183432, 183439, 117911, 117918, 117934, 117940, 183480, 150713, 150715, 117947, 150719, 150720, 183487, 117954, 150722, 183494, 150729, 150731, 150735, 150737, 117971, 183509, 150743, 183511, 183519, 117992, 117993, 117994, 183530, 117999, 118008, 118009, 150777, 183546, 118012, 183551, 183554, 183558, 183559, 150793, 150803, 150804, 183574, 183577, 118044, 118057, 118060, 118061, 183598, 118074, 118076, 118077, 150851, 118084, 150852, 183626, 183627, 118101, 150871, 150879, 183649, 118120, 150891, 150893, 150897, 183670, 183672, 118138, 183688, 183694, 183700, 150946, 118179, 183714, 183723, 150956, 183724, 118196, 183741, 183743, 118210, 150983, 150989, 183757, 150993, 183762, 118227, 183768, 151002, 183772, 118243, 118249, 183788, 118253, 151021, 151032, 183804, 151037, 183806, 151045, 151048, 151051, 183820, 151058, 183834, 183835, 151069, 151071, 118308, 118309, 183845, 183850, 151088, 151092, 151104, 151108, 183879, 118348, 118351, 151122, 118358, 151131, 151132, 183906, 151143, 118377, 118380, 183917, 118384, 183924, 183934, 183935, 183936, 183943, 151178, 183957, 183958, 151192, 183964, 183965, 118430, 118434, 183978, 118455, 183993, 118467, 118470, 118475, 118481, 118499, 118505, 118507, 118508, 118510, 151282, 118522, 184066, 151299, 151300, 184071, 151305, 184076, 151312, 151313, 151314, 151316, 151318, 151329, 184101, 151336, 118589, 118593, 151362, 118594, 118597, 184135, 151372, 118610, 118625, 184164, 184168, 118636, 118646, 118649, 118654, 118655, 151424, 151425, 184194, 151427, 184200, 118666, 118669, 151439, 151449, 151454, 184223, 151469, 151470, 184244, 184247, 118711, 151480, 118717, 184266, 118731, 118735, 118744, 151517, 184289, 118754, 118755, 151523, 118760, 118768, 151537, 151547, 151554, 151566, 184334, 118799, 118802, 118805, 184345, 151578, 151584, 151594, 118833, 184380, 151613, 118847, 151618, 118864, 184401, 118866, 151632, 184405, 118871, 184409, 118874, 151644, 184419, 151654, 118886, 118889, 151677, 184460, 118924, 184463, 184464, 184470, 151711, 184481, 184485, 151723, 151725, 184495, 151727, 118963, 118970, 184508, 184517, 118987, 151764, 119001, 184554, 184564, 151797, 184565, 119031, 184566, 184571, 184606, 184608, 151840, 119086, 119095, 184633, 151869, 119101, 119104, 151880, 184658, 151898, 151901, 184670, 184671, 151904, 151905, 119136, 119150, 119167, 184704, 119171, 119173, 151958, 151959, 119192, 151964, 184735, 119204, 151973, 184746, 119214, 184751, 151984, 119215, 184750, 151990, 151992, 184760, 184765, 184770, 119241, 152012, 152015, 184784, 184788, 119256, 184792, 119260, 119261, 152030, 152031, 152037, 119280, 184819, 184828, 152063, 152065, 152075, 119308, 152085, 152092, 184860, 152094, 152101, 119334, 184876, 184880, 152114, 152117, 152118, 152121, 119356, 184900, 119375, 152147, 152148, 152158, 184927, 119391, 119398, 119400, 184941, 152185, 119420, 184966, 184970, 119437, 119454, 119455, 184992, 119461, 152232, 119470, 185006, 119477, 185018, 152251, 152254, 152264, 119496, 152267, 185036, 152271, 152274, 185045, 185057, 185069, 185071, 119535, 185074, 152308, 152315, 119547, 152335, 119573, 185110, 152354, 152358, 119591, 152364, 152365, 119599, 152434, 152436, 185204, 119671, 119672, 185211, 185220, 119693, 119699, 152468, 119704, 119708, 152478, 119710, 185263, 152497, 152507, 119768, 119778, 152550, 185321, 119803, 119807, 185344, 185358, 185363, 152601, 152612, 119848, 119853, 119858, 152627, 119860, 152631, 185400, 185409, 119878, 152649, 119882, 119897, 185436, 152670, 152675, 152694, 152696, 152701, 119943, 185480, 152718, 185488, 119960, 119962, 152737, 119980, 185518, 185520, 152754, 152759, 152760, 119993, 185531, 152764, 152763, 120000, 120002, 185539, 152778, 185549, 120014, 185552, 120022, 185562, 120027, 152797, 120029, 185574, 152816, 120051, 185603, 185604, 120074, 185611, 185610, 120075, 152847, 120099, 185638, 120102, 185643, 185645, 120113, 152886, 185660, 152893, 152895, 152915, 120148, 185687, 152925, 152929, 120164, 120165, 152934, 152936, 185708, 152943, 185715, 185717, 185723, 152957, 152964, 185747, 120212, 152985, 120218, 185753, 185759, 185761, 185762, 152998, 120233, 185771, 153013, 120251, 185789, 185797, 153031, 120267, 153036, 153044, 185818, 185820, 120290, 153059, 120292, 185833, 153067, 120306, 185846, 153091, 120325, 153105, 185889, 153128, 153129, 153131, 153138, 153141, 153143, 120375, 153146, 185919, 185927, 120392, 120396, 153166, 153167, 120406, 153175, 120414, 185953, 185954, 120424, 120433, 153202, 153207, 185975, 185978, 120446, 185988, 120454, 120455, 185993, 185994, 185998, 153233, 186002, 153237, 186009, 153246, 186029, 186034, 153272, 120508, 153281, 186057, 153292, 120540, 153319, 153325, 153331, 120569, 186106, 120575, 186113, 120585, 186123, 153364, 186136, 153377, 153384, 186155, 186161, 186162, 153394, 153403, 153408, 120640, 186178, 153410, 186185, 153420, 153422, 120655, 120663, 186203, 186208, 186209, 120675, 153461, 186236, 120702, 153470, 186245, 186247, 186251, 120720, 153500, 153507, 186278, 186285, 186286, 120754, 153523, 153527, 120762, 186298, 120766, 186308, 120778, 186314, 186316, 153550, 153555, 186326, 120791, 186333, 153570, 186338, 120806, 186343, 186355, 186361, 153596, 186366, 120831, 120838, 186375, 186377, 186386, 153621, 120853, 186391, 120861, 153632, 120867, 153645, 186413, 186418, 153651, 186419, 153661, 120896, 120901, 153675, 153679, 186454, 186455, 120923, 153694, 186464, 120949, 186490, 120956, 153726, 153729, 120965, 120967, 153748, 186518, 153754, 186531, 153765, 153766, 153777, 153781, 186549, 121019, 186557, 121027, 153803, 121036, 121035, 121040, 153808, 186586, 186587, 186590, 186604, 186618, 186624, 153860, 121109, 153877, 186654, 153896, 186682, 153922, 121157, 186693, 186695, 153928, 153929, 153933, 153935, 186707, 186719, 186726, 153969, 153970, 121208, 186750, 121215, 153984, 186754, 121237, 121238, 186775, 154008, 154015, 186787, 154021, 121263, 186803, 121273, 154052, 186826, 121292, 121293, 121316, 121321, 121324, 121329, 154097, 121338, 121340, 121343, 186885, 121354, 186892, 121357, 186894, 186895, 186904, 121373, 154144, 121377, 154148, 154149, 154150, 154154, 186925, 186933, 186939, 154173, 186941, 154175, 121411, 186952, 186956, 154196, 186966, 154202, 121435, 186981, 186983, 154228, 154230, 154232, 154238, 121480, 121481, 154252, 121490, 154263, 121497, 154266, 187037, 154276, 121515, 187054, 154293, 187062, 121528, 187079, 187083, 154318, 187089, 154322, 154323, 154325, 187114, 187135, 187137, 121604, 154374, 121611, 187156, 154395, 121637, 154411, 154417, 121662, 187202, 187213, 121681, 154451, 187221, 154473, 154474, 187242, 121709, 154486, 154492, 187263, 187267, 154506, 121738, 187278, 187279, 121747, 187284, 187288, 121758, 154530, 121765, 187307, 187311, 154545, 121788, 187327, 187328, 121800, 154571, 121812, 154584, 154585, 154590, 154592, 121832, 187374, 187375, 187377, 187379, 154612, 121846, 121847, 154617, 187399, 154633, 154638, 187410, 187413, 121878, 154647, 154651, 187424, 187428, 121897, 154672, 187447, 154686, 187469, 187471, 154703, 187477, 121947, 121949, 187493, 187498, 187501, 187502, 154743, 187511, 121979, 187516, 187520, 154753, 187522, 154757, 187526, 154761, 121994, 154767, 121999, 187540, 187542, 187547, 154780, 122020, 187559, 154794, 154799, 154811, 187592, 187606, 122075, 154854, 154856, 122088, 187626, 122093, 187638, 122104, 122116, 154886, 187654, 187660, 122139, 122154, 187690, 122161, 154930, 187708, 122180, 122185, 187728, 122193, 122196, 154972, 187743, 154976, 122209, 122218, 154990, 154993, 187761, 122228, 122231, 155003, 187772, 187782, 122249, 122267, 155046, 187828, 155065, 187834, 187837, 155071, 155074, 187852, 187883, 187893, 155131, 122371, 187912, 187915, 187921, 187927, 187931, 122400, 122419, 187966, 187968, 187971, 155207, 187978, 155212, 155219, 122451, 155230, 155246, 188015, 122479, 155256, 122488, 155261, 122498, 155273, 155274, 188043, 188046, 122518, 155287, 155290, 188059, 122523, 155301, 188072, 155310, 155311, 155312, 155315, 155320, 122561, 155331, 155333, 188101, 122570, 122578, 155347, 155348, 122595, 155375, 122614, 188151, 122615, 122616, 188153, 188155, 188158, 122627, 188166, 122631, 188169, 188181, 188185, 122654, 155426, 155429, 155431, 188200, 188202, 188204, 188205, 122672, 188210, 188227, 122697, 188236, 155470, 188240, 122706, 155486, 155494, 188272, 155505, 188276, 188278, 155513, 155521, 122755, 122771, 155542, 122777, 122779, 188327, 155561, 122807, 155577, 155583, 155588, 188361, 155595, 155599, 122836, 122841, 122842, 188377, 188383, 155618, 122850, 122851, 155625, 122863, 122879, 155647, 155650, 188432, 122903, 122908, 155687, 122930, 188469, 188481, 122954, 188492, 122960, 122961, 155731, 122979, 122980, 155760, 155761, 188532, 122997, 155773, 155777, 188547, 155780, 188553, 188554, 123023, 123026, 155797, 155798, 188566, 188571, 155805, 188583, 123064, 123073, 123074, 123075, 123081, 188623, 155865, 155874, 123110, 155880, 188651, 155885, 188653, 155894, 155906, 188679, 155913, 123147, 188685, 155917, 123159, 188700, 155935, 188704, 123171, 155940, 123178, 188719, 123195, 123196, 123210, 155980, 155985, 123226, 123230, 123233, 123243, 123252, 188791, 156024, 123258, 188802, 123268, 188805, 156052, 188829, 188840, 123311, 123313, 188856, 156092, 123324, 123345, 188882, 123357, 156131, 156141, 188912, 156152, 123393, 156164, 123399, 188936, 123402, 123404, 188944, 156178, 188946, 156180, 156182, 123420, 156192, 156200, 188976, 188979, 123444, 156221, 189000, 189007, 189008, 123475, 189016, 123481, 189026, 123505, 156276, 123509, 123511, 156284, 156286, 156288, 189062, 156302, 156310, 189083, 189094, 123558, 156328, 123561, 123562, 123563, 156340, 123581, 189128, 156374, 123609, 123610, 156381, 189154, 189155, 123618, 189157, 156386, 156393, 156402, 156412, 123645, 156417, 189186, 156419, 123652, 123663, 189205, 123670, 123674, 156446, 156456, 123701, 156471, 156475, 189248, 156481, 123717, 189255, 123722, 189258, 189259, 123723, 156498, 189267, 189276, 123743, 156517, 189288, 123755, 156527, 189298, 189300, 123764, 123768, 123771, 189311, 123776, 123777, 156556, 156564, 156573, 156578, 189349, 189351, 123816, 189380, 189385, 189389, 123854, 123855, 156625, 123860, 156628, 123866, 156638, 123872, 156649, 123884, 189421, 123885, 156655, 156657, 123892, 189435, 156674, 156682, 156694, 156698, 156706, 156708, 123946, 189489, 189494, 156728, 189497, 156733, 123966, 189505, 189506, 156739, 123974, 123980, 156760, 124007, 156784, 124029, 189565, 124032, 156801, 156806, 156813, 124048, 189585, 156820, 156822, 189591, 124058, 124062, 189599, 156837, 156839, 156840, 156841, 189613, 189614, 124080, 156849, 124083, 189627, 156862, 124107, 189644, 189655, 189657, 156893, 156896, 189667, 189668, 189669, 124140, 124142, 189679, 124154, 156925, 189703, 156950, 189721, 156957, 156958, 189736, 189751, 156983, 124216, 189754, 156992, 124231, 189786, 189788, 124253, 124258, 157044, 124277, 124281, 157051, 189821, 157056, 189825, 124289, 124292, 189830, 124301, 124306, 189850, 124323, 124324, 189861, 124327, 157100, 157106, 124341, 124343, 124353, 189894, 124365, 157136, 124372, 189920, 189921, 189923, 157156, 124390, 124391, 124412, 189952, 157185, 124423, 157213, 189985, 189995, 124461, 124476, 124482, 124486, 157255, 157264, 124498, 124500, 157269, 190038, 124503, 190053, 124517, 124522, 190062, 124531, 157299, 157304, 124542, 190094, 190096, 124565, 157334, 124569, 157358, 157361, 190129, 157364, 190134, 157369, 157378, 190149, 124616, 124621, 157390, 190162, 157399, 157400, 190170, 157405, 124639, 190178, 190179, 124649, 157418, 157417, 190190, 157423, 190204, 124676, 190218, 157460, 157464, 124697, 157468, 157470, 124711, 157480, 157483, 190261, 124728, 190269, 190272, 190278, 190281, 190283, 124749, 157518, 190286, 190290, 157522, 124756, 157533, 190319, 124785, 190329, 157563, 124796, 124798, 124808, 190351, 190365, 157614, 157618, 157620, 124856, 190393, 157626, 190395, 157634, 124871, 190411, 157646, 124881, 157654, 157656, 190445, 124917, 190453, 190457, 124922, 190460, 124924, 157694, 190464, 190468, 124934, 124945, 124950, 190488, 190494, 124958, 157729, 124963, 157735, 157736, 157738, 124971, 124972, 157741, 190512, 157750, 124983, 124989, 157772, 190551, 190553, 190558, 157803, 190572, 125050, 157818, 190591, 125056, 157828, 157829, 125062, 157831, 190600, 190603, 190606, 157841, 157846, 190620, 190624, 125092, 190630, 157867, 157868, 190637, 190635, 190648, 157880, 125118, 157887, 190661, 190666, 190677, 157921, 125154, 125157, 125163, 157932, 190701, 125172, 157942, 190715, 157956, 125190, 157960, 190733, 190738, 190740, 157974, 125212, 125214, 190763, 157997, 125230, 157999, 125243, 158014, 158021, 158024, 158035, 125284, 158054, 158078, 190848, 158080, 158086, 158089, 125333, 190873, 190885, 125368, 158136, 125372, 190909, 158142, 158150, 125385, 125387, 190930, 190933, 125398, 125399, 158174, 125407, 158178, 125411, 125415, 125417, 190956, 190957, 158192, 158215, 158216, 125448, 125450, 158221, 125465, 125469, 158253, 158278, 158283, 191052, 191054, 158288, 125521, 191069, 191095, 158327, 191108, 125574, 158343, 191113, 191115, 191117, 158350, 125583, 125586, 125590, 125594, 158365, 125598, 191133, 158373, 191148, 125616, 158385, 158386, 158392, 191162, 125626, 191165, 191171, 191205, 191208, 191210, 158442, 158453, 125689, 125693, 191230, 158463, 191232, 158468, 125701, 158470, 158471, 125721, 158496, 125731, 158504, 191273, 191275, 125758, 191305, 191308, 125774, 191315, 158550, 158560, 158561, 158564, 158571, 158581, 125824, 158596, 158604, 125843, 191389, 158621, 191403, 191404, 191408, 191415, 191418, 125900, 158677, 191454, 158688, 191467, 191473, 158706, 125939, 158711, 191482, 158715, 125948, 125949, 158722, 125960, 191497, 158728, 191512, 125983, 158752, 191521, 125992, 191533, 158767, 126007, 191543, 126011, 191560, 126034, 126035, 126037, 158807, 126041, 158810, 191578, 158813, 191592, 158829, 126065, 191601, 191606, 191614, 158855, 191631, 126100, 191639, 158873, 191645, 126116, 158896, 126132, 158904, 126151, 158919, 158924, 126157, 126156, 191701, 126165, 191706, 191708, 158944, 126179, 191721, 158956, 191724, 191729, 191737, 191738, 126207, 158989, 191759, 158993, 191764, 191791, 159023, 159025, 126264, 159035, 159038, 191814, 159048, 191819, 126287, 126289, 191829, 126294, 191832, 126302, 126309, 159079, 159095, 126332, 191872, 191875, 126341, 191879, 159113, 191884, 159117, 191887, 126360, 191898, 159132, 159137, 191907, 159140, 126378, 191918, 191919, 191924, 159157, 159159, 191929, 126394, 159163, 126393, 191940, 191950, 191971, 126435, 159209, 126442, 191978, 191981, 191991, 191993, 191996, 159231, 126470, 126474, 159247, 126487, 192025, 192026, 192031, 126510, 192053, 192055, 159288, 159295, 159298, 192084, 192098, 192105, 126576, 126586, 126593, 159362, 192130, 159364, 192139, 126608, 159378, 192151, 126620, 126626, 159395, 126627, 126629, 159400, 192175, 192179, 159442, 192214, 126681, 126688, 159458, 192230, 159469, 192240, 192243, 159475, 192249, 159491, 159495, 192269, 192274, 192277, 192292, 126761, 126763, 159539, 192309, 192310, 126773, 126778, 126783, 159554, 126790, 126791, 192327, 192329, 126801, 126804, 126807, 159575, 126815, 159586, 159591, 159596, 192369, 192374, 192377, 126844, 192382, 192384, 192399, 126871, 126877, 126883, 159655, 159666, 159680, 192452, 192457, 159692, 192462, 192475, 192480, 126954, 126955, 192494, 192495, 192502, 159737, 126972, 159753, 192529, 159771, 127008, 127021, 127024, 159799, 192567, 159801, 159807, 192580, 159814, 127049, 192591, 192602, 192606, 159843, 159846, 159850, 127082, 192620, 127088, 192625, 127096, 159867, 159868, 192635, 127105, 127106, 159875, 127108, 127117, 127118, 127133, 127137, 159907, 159913, 192688, 159924, 192695, 192698, 127164, 192701, 159938, 159943, 127180, 159950, 127184, 192720, 159959, 127195, 127198, 159967, 192741, 159974, 192743, 127208, 159980, 159989, 127230, 160005, 127238, 127239, 160010, 192782, 192784, 127249, 127259, 160027, 127266, 192809, 127282, 160055, 127294, 127296, 192833, 160072, 192849, 160095, 160100, 192876, 127344, 160118, 192891, 160126, 127361, 160135, 160159, 127392, 192929, 160163, 160172, 160173, 192940, 127408, 127414, 127420, 160188, 127428, 192969, 160208, 160210, 192987, 193011, 193017, 127483, 127489, 127491, 127501, 160270, 127503, 127507, 193046, 127513, 160285, 127517, 193060, 160303, 193076, 127542, 127545, 160318, 127552, 193089, 127559, 193103, 160337, 193114, 160366, 193136, 127603, 127604, 193153, 193158, 127622, 127630, 193168, 193172, 127637, 193175, 193178, 193190, 127657, 127662, 127664, 193207, 193216, 160448, 193222, 193223, 160456, 193230, 193235, 193241, 127713, 127716, 193254, 193258, 160505, 193278, 127742, 160517, 193294, 193298, 127775, 127779, 160553, 127788, 160556, 160561, 193332, 127809, 160577, 127815, 193351, 127823, 193360, 160593, 193362, 160594, 160591, 127829, 160598, 193368, 160605, 160607, 127839, 127846, 127849, 160620, 127854, 193399, 193401, 193403, 127867, 193405, 193408, 127884, 160654, 127886, 193434, 160674, 127911, 127912, 127915, 127920, 193457, 127924, 193463, 127930, 160705, 160709, 160726, 160731, 127965, 193509, 160744, 127977, 127978, 127985, 160760, 160763, 193535, 160773, 160776, 160779, 128013, 160784, 193555, 193565, 160797, 193575, 128039, 193577, 128041, 160813, 160818, 160823, 193593, 160825, 160831, 160833, 193602, 160834, 160836, 193605, 128069, 160847, 160849, 193620, 193621, 160853, 128096, 160874, 128115, 193651, 128116, 193659, 160894, 128131, 160909, 128144, 128156, 160928, 160929, 128162, 160937, 160945, 160946, 193732, 160967, 193760, 160992, 160998, 161008, 161011, 128243, 161013, 128245, 161020, 161030, 193799, 161033, 193803, 128268, 128275, 193816, 128281, 161048, 193820, 128289, 193828, 128294, 193833, 161069, 193838, 193848, 161088, 193857, 193858, 128326, 193864, 161097, 161099, 161109, 193878, 128344, 193894, 128362, 193911, 161146, 161149, 193931, 161173, 161174, 193941, 193948, 161182, 128416, 161187, 161188, 161200, 193970, 193971, 161204, 161208, 193978, 128444, 128448, 193989, 161228, 128465, 161234, 161236, 161239, 128475, 161253, 161257, 128490, 194040, 128514, 194053, 161287, 161290, 194067, 161301, 128544, 128548, 194096, 161333, 128573, 161341, 194111, 194112, 194116, 194117, 128581, 161352, 128587, 194126, 194131, 194132, 128596, 128598, 161367, 194154, 128632, 161405, 128642, 128644, 128647, 161415, 194189, 161430, 194201, 161439, 194216, 161448, 161449, 128684, 128694, 161474, 161476, 161492, 194263, 128732, 194269, 194275, 161507, 128741, 161511, 128753, 194291, 161526, 194301, 128775, 161548, 128783, 194322, 161558, 128810, 128811, 194353, 161586, 194358, 194360, 128836, 161609, 128842, 194379, 161614, 161622, 161623, 161625, 128857, 128859, 161632, 194406, 194418, 194420, 128886, 161654, 161656, 161665, 128902, 161671, 128904, 194446, 194451, 128916, 194463, 194465, 128930, 128948, 161722, 194500, 194507, 161742, 161743, 161747, 128986, 128987, 128989, 194525, 161764, 161772, 194541, 161790, 194561, 129029, 194573, 194575, 129040, 194577, 194576, 194579, 129045, 161814, 194584, 194585, 129063, 194604, 161839, 129076, 161845, 194618, 161851, 129084, 194621, 194626, 194630, 194637, 161870, 194640, 129104, 194641, 129109, 161880, 194654, 161887, 129133, 194671, 129142, 129146, 161921, 161926, 161927, 194696, 129162, 194703, 129168, 161940, 129176, 194713, 194717, 129189, 161958, 194736, 161970, 161972, 194759, 129223, 162005, 129240, 129244, 129246, 162016, 162017, 194788, 162021, 162022, 194794, 162031, 194799, 194801, 129275, 194814, 194833, 129297, 194844, 194847, 194859, 194864, 194865, 129330, 129332, 162108, 162109, 194892, 194897, 162129, 194906, 129372, 129382, 194937, 194948, 194958, 162193, 194961, 162204, 129444, 162219, 162221, 129453, 162231, 129467, 129468, 195007, 195009, 162248, 162252, 162259, 129495, 195041, 129508, 129511, 195066, 195073, 162320, 129555, 162330, 195101, 129565, 162343, 195111, 162358, 195128, 129611, 162379, 162383, 129616, 162391, 162392, 195161, 129630, 162411, 195182, 162416, 195195, 162435, 162438, 162439, 162441, 129674, 129675, 162446, 162452, 129695, 162463, 129699, 129713, 162482, 195256, 129721, 195260, 195271, 162506, 129749, 195294, 162533, 129767, 162542, 129776, 162546, 129780, 162550, 129787, 162564, 129798, 162568, 129802, 195344, 129810, 162579, 195348, 195349, 195350, 129821, 129848, 195385, 162617, 195384, 129856, 129861, 162633, 162636, 195408, 162646, 129882, 195419, 195421, 162663, 162668, 195438, 195451, 129916, 129917, 162688, 129935, 162706, 195487, 129955, 162726, 195502, 162742, 195528, 162765, 162768, 195552, 130019, 130026, 130027, 195565, 162803, 130039, 195583, 195595, 130065, 195615, 162850, 195624, 130091, 130096, 162866, 130101, 130108, 162878, 195676, 130144, 162921, 130154, 195692, 162930, 130164, 195706, 162940, 130186, 130187, 130193, 162969, 195743, 162999, 163003, 195775, 130242, 195791, 195795, 195809, 163048, 195821, 195834, 195836, 195850, 195853, 130327, 130330, 163109, 195878, 163110, 195884, 130358, 130360, 163133, 195903, 195904, 163151, 195922, 130390, 163158, 163159, 195934, 130399, 195936, 130401, 195942, 163180, 130412, 163186, 163189, 195958, 130423, 195962, 163200, 130435, 163205, 130447, 195984, 130453, 195990, 130457, 130458, 163227, 163225, 163230, 163247, 163254, 130495, 196034, 196036, 196043, 130510, 196047, 130520, 130527, 163295, 196066, 163305, 196074, 130539, 163308, 163307, 163314, 130553, 163321, 163324, 196098, 130564, 130579, 163354, 196123, 130592, 163361, 196132, 163366, 196135, 163369, 196151, 130623, 163392, 196160, 163404, 163407, 163420, 130653, 196199, 196208, 163441, 163440, 196220, 130697, 163466, 163469, 130710, 130711, 163480, 163481, 196250, 163504, 196275, 196277, 130745, 163513, 196294, 130762, 163530, 163539, 130775, 130777, 163553, 130787, 196324, 163560, 196328, 196335, 196336, 130806, 130819, 130821, 196360, 130828, 163600, 196373, 163614, 163617, 163621, 130855, 130861, 163634, 163635, 196409, 196412, 163645, 163647, 163652, 130887, 196423, 130889, 196446, 196459, 130923, 196464, 196473, 130947, 130949, 196486, 196489, 130955, 163724, 163736, 130970, 196506, 163742, 130975, 196517, 163753, 163754, 196522, 196524, 163762, 130998, 163777, 131022, 131026, 163810, 163811, 163813, 131050, 163821, 163826)
order by c.hadm_id, c.charttime
"""

bloodpre = pd.read_sql_query(query,con)
bloodpre.to_csv(exportdir+'bloodpressure.csv',index=False,sep='|')
bloodpre.head()

In [ ]:
had_total = bloodpre.hadm_id.unique()
print(len(had_total))

In [ ]:
filepath= "/Users/yan/processed_data/processedFrames.pkl"
OutputFrames = pickle.load(open(filepath, 'rb'))

In [ ]:
key_out = OutputFrames.keys()

In [ ]:
def buildTimeFrame(table_h, delta):
    # Get admit and discharge time in numeric form, round down/up respectively to the nearest hour:
    
    start =  pd.to_datetime(table_h.admit_time.unique().item())
    start -= dt.timedelta(minutes=start.minute, seconds=start.second, microseconds=start.microsecond)
    end = pd.to_datetime(table_h.discharge_time.unique().item())
    end -= dt.timedelta(minutes=end.minute, seconds=end.second, microseconds=end.microsecond)
    end += dt.timedelta(hours=1)
    
    times = []
    curr = start
    while curr < end:
        times.append(curr)
        curr += delta
    timeFrame = pd.DataFrame(data={'timestamp': times}, index=times)
    return timeFrame

In [ ]:
misc_list = ['Arterial Blood Pressure systolic','Arterial Blood Pressure diastolic','Arterial Blood Pressure mean']

In [ ]:
for key, chartFrame in OutputFrames.items():
    if bloodpre[bloodpre.hadm_id == key].empty == False:
        measures_table = bloodpre[bloodpre.hadm_id == key]
        for v in (misc_list):
            chartFrame[v] = np.nan
            vitals_v = measures_table[(measures_table.label == v)].sort_values(by='charttime')
            vitals_v.set_index('charttime',inplace=True,drop=False)
            vitals_v = vitals_v.resample('1h').mean().fillna(method="ffill")  
            vitals_v['timestamp'] = vitals_v.index
            for t in chartFrame.timestamp:
                if vitals_v[vitals_v.timestamp == t].empty == False:
                    chartFrame.loc[chartFrame.timestamp == t,v] = vitals_v[vitals_v.timestamp == t].valuenum.item()    

In [ ]:
outputname = "/Users/yan/processed_data/processedFrames.pkl"
pickle.dump(OutputFrames, open(outputname,'wb'))

In [ ]:
########## add weight in the data when the weight is missing ######################

In [2]:
filepath= "/Users/yan/processed_data/processedFrames.pkl"
OutputFrames = pickle.load(open(filepath, 'rb'))

In [3]:
a = OutputFrames[147593]

In [4]:
#a.to_csv('/Users/yan/data_mcvent/example_114691bp.csv', index = False)

In [30]:
b = OutputFrames[131262]

In [6]:
#b.to_csv('/Users/yan/data_mcvent/example_131262bp.csv', index = False)

In [4]:
tables='/Users/yan/processed_data/allTables.pkl'
adms_df, vent_df, measures_df, seds_df, sbt_df = pickle.load(open(tables, 'rb'))

In [5]:
for key, chartFrame in OutputFrames.items():
    if pd.isnull(chartFrame['Admission Weight (Kg)'].iloc[0]) == True:
        measures_table = measures_df[measures_df.hadm == key]
        measures_table = measures_table[measures_table.label == 'Admission Weight (Kg)']
        if measures_table.empty == False:
            weight_value = measures_table.value.head(1).item()    
#            chartFrame['Admission Weight (Kg)'] = weight_value
            for i in chartFrame.index:
                if pd.isnull(chartFrame.loc[i, 'Admission Weight (Kg)']) == True:
                    chartFrame.loc[i, 'Admission Weight (Kg)'] = weight_value            

In [10]:
outputname = "/Users/yan/processed_data/processedFrames.pkl"
pickle.dump(OutputFrames, open(outputname,'wb'))

In [ ]:
###deal with blood pressure missing data if Arterial Blood Pressure is missing, use Non-invasive blood pressure instead#####

In [ ]:
for key, chartFrame in OutputFrames.items():
    chartFrame = chartFrame.fillna(method = 'ffill').fillna(method = 'bfill')
    OutputFrames[key] = chartFrame

In [ ]:
for key, chartFrame in OutputFrames.items():
    if pd.isnull(chartFrame['Arterial Blood Pressure systolic'].iloc[0]) == True:
        if pd.isnull(chartFrame['Non Invasive Blood Pressure systolic'].iloc[0]) == False:
            for i in chartFrame.index:
                bp_value = chartFrame.loc[i, 'Non Invasive Blood Pressure systolic']
                chartFrame.loc[i, 'Arterial Blood Pressure systolic'] = bp_value 

In [ ]:
for key, chartFrame in OutputFrames.items():
    if pd.isnull(chartFrame['Arterial Blood Pressure diastolic'].iloc[0]) == True:
        if pd.isnull(chartFrame['Non Invasive Blood Pressure diastolic'].iloc[0]) == False:
            for i in chartFrame.index:
                bp_value = chartFrame.loc[i, 'Non Invasive Blood Pressure diastolic']
                chartFrame.loc[i, 'Arterial Blood Pressure diastolic'] = bp_value 

In [ ]:
for key, chartFrame in OutputFrames.items():
    if pd.isnull(chartFrame['Arterial Blood Pressure mean'].iloc[0]) == True:
        if pd.isnull(chartFrame['Non Invasive Blood Pressure mean'].iloc[0]) == False:
            for i in chartFrame.index:
                bp_value = chartFrame.loc[i, 'Non Invasive Blood Pressure mean']
                chartFrame.loc[i, 'Arterial Blood Pressure mean'] = bp_value  

In [ ]:
outputname = "/Users/yan/processed_data/processedFrames.pkl"
pickle.dump(OutputFrames, open(outputname,'wb'))

In [ ]:
a = OutputFrames[131570]
a.to_csv('/Users/yan/data_mcvent/example_131570.csv', index = False)

In [ ]:
####### ventilator mode is different from other vital signs, it is meaningless to average the value of ventilator mode
###  as it is categorical not continuous.

In [ ]:
tables='/Users/yan/processed_data/allTables.pkl'
adms_df, vent_df, measures_df, seds_df, sbt_df = pickle.load(open(tables, 'rb'))

In [ ]:
for key, chartFrame in OutputFrames.items():
    measures_table = measures_df[measures_df.hadm == key]
    chartFrame['Ventilator Mode'] = np.nan
    vitals_v = measures_table[(measures_table.label == 'Ventilator Mode')].sort_values(by='charttime')
    vitals_v.set_index('charttime',inplace=True,drop=False)
    vitals_v = vitals_v.resample('1h').last().fillna(method="ffill")  
    vitals_v['timestamp'] = vitals_v.index
    for t in chartFrame.timestamp:
        if vitals_v[vitals_v.timestamp == t].empty == False:
            chartFrame.loc[chartFrame.timestamp == t,'Ventilator Mode'] = vitals_v[vitals_v.timestamp == t].value.item()                    

In [ ]:
for key, chartFrame in OutputFrames.items():
    chartFrame = chartFrame.fillna(method = 'ffill').fillna(method = 'bfill')
    OutputFrames[key] = chartFrame

In [ ]:
outputname = "/Users/yan/processed_data/processedFrames.pkl"
pickle.dump(OutputFrames, open(outputname,'wb'))